In [1]:
from icecream import ic
from pprint import pprint
import regex as re

In [6]:
text = "hello louis hey 🤐!!!! how is the world ?"
# text = "🤐"
# text = "hello world!"
# text = "é"
tokens = text.encode("utf-8")
tokens
ic(int(tokens[0]))
# len(list(map(int, tokens))), len(text)
print([int(x) for x in tokens]);

ic| int(tokens[0]): 104


[104, 101, 108, 108, 111, 32, 108, 111, 117, 105, 115, 32, 104, 101, 121, 32, 240, 159, 164, 144, 33, 33, 33, 33, 32, 104, 111, 119, 32, 105, 115, 32, 116, 104, 101, 32, 119, 111, 114, 108, 100, 32, 63]


In [3]:
def get_stats(ids):
    counts = {}
    for pair in zip(ids, ids[1:]):
        counts[pair] = counts.get(pair, 0) + 1
    return counts


stats = get_stats(tokens)
# sorted(((v, k) for k, v in counts.items()), reverse=True)
stats

{(104, 101): 3,
 (101, 108): 1,
 (108, 108): 1,
 (108, 111): 2,
 (111, 32): 1,
 (32, 108): 1,
 (111, 117): 1,
 (117, 105): 1,
 (105, 115): 2,
 (115, 32): 2,
 (32, 104): 2,
 (101, 121): 1,
 (121, 32): 1,
 (32, 240): 1,
 (240, 159): 1,
 (159, 164): 1,
 (164, 144): 1,
 (144, 33): 1,
 (33, 33): 3,
 (33, 32): 1,
 (104, 111): 1,
 (111, 119): 1,
 (119, 32): 1,
 (32, 105): 1,
 (32, 116): 1,
 (116, 104): 1,
 (101, 32): 1,
 (32, 119): 1,
 (119, 111): 1,
 (111, 114): 1,
 (114, 108): 1,
 (108, 100): 1,
 (100, 32): 1,
 (32, 63): 1}

In [4]:
top_pair = max(stats, key=stats.get)
top_pair

(104, 101)

In [5]:
def merge(ids, pair, new_idx):
    new_ids = []
    i = 0
    while i < len(ids):
        if i < len(ids) - 1 and ids[i] == pair[0] and ids[i + 1] == pair[1]:
            new_ids.append(new_idx)
            i += 2
        else:
            new_ids.append(ids[i])
            i += 1

    return new_ids


merge([1, 2, 5, 6, 7, 5, 6], (5, 6), 99)

merge(tokens, top_pair, 256)

[256,
 108,
 108,
 111,
 32,
 108,
 111,
 117,
 105,
 115,
 32,
 256,
 121,
 32,
 240,
 159,
 164,
 144,
 33,
 33,
 33,
 33,
 32,
 104,
 111,
 119,
 32,
 105,
 115,
 32,
 116,
 256,
 32,
 119,
 111,
 114,
 108,
 100,
 32,
 63]

In [6]:
vocab_size = 276
num_merges = vocab_size - 256
ids = list(tokens)
merges = {}

for i in range(num_merges):
    stats = get_stats(ids)
    if len(stats) > 0:
        top_pair = max(stats, key=stats.get)
        idx = 256 + i
        ids = merge(ids, top_pair, idx)
        merges[top_pair] = idx
    else:
        break
merges

{(104, 101): 256,
 (33, 33): 257,
 (108, 111): 258,
 (105, 115): 259,
 (259, 32): 260,
 (256, 108): 261,
 (261, 258): 262,
 (262, 32): 263,
 (263, 258): 264,
 (264, 117): 265,
 (265, 260): 266,
 (266, 256): 267,
 (267, 121): 268,
 (268, 32): 269,
 (269, 240): 270,
 (270, 159): 271,
 (271, 164): 272,
 (272, 144): 273,
 (273, 257): 274,
 (274, 257): 275}

In [7]:
vocab = {idx: bytes([idx]) for idx in range(256)}
for (p0, p1), idx in merges.items():
    # ic(p0, p1)
    vocab[idx] = vocab[p0] + vocab[p1]


def decode(ids: list) -> str:
    tokens = b"".join(vocab[idx] for idx in ids)
    txt = tokens.decode("utf-8", errors="replace")
    return txt

In [8]:
def encode(text):
    tokens = list(text.encode("utf-8"))
    # ic(tokens)
    while len(tokens) > 1:
        stats = get_stats(tokens)
        # ic(stats)
        pair = min(stats, key=lambda p: merges.get(p, float("inf")))
        # ic(merges)
        # ic(pair)
        if pair not in merges:
            break
        idx = merges[pair]
        # ic(idx)
        tokens = merge(tokens, pair, idx)
        # ic(tokens)
    return tokens


decode(encode("Hello lo !! you you"))
#encode("h")

'Hello lo !! you you'

## Pre-tokenizer

In [9]:
pat = re.compile(
    r"""'s|'t|'re|'ve|'m|'ll|'d| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+"""
)
pat.findall("hello World!, You're")

['hello', ' World', '!,', ' You', "'re"]